In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

# 데이터 전처리 (Normalization)
transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                transforms.RandomCrop(32, padding=4),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# CIFAR-10 데이터 로드
# 32*32 60000개 컬러 이미지
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

# 간단한 CNN 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(3, 128, 3,padding=1) # in_channels, out_channels, kernel_size
        self.conv2 = nn.Conv2d(128, 256, 3,padding=1)
        self.conv3 = nn.Conv2d(256, 512, 3,padding=1)
        self.conv4 = nn.Conv2d(512, 1024, 3)
        self.fc1 = nn.Linear(1024 * 1 * 1, 128)
        #self.fc2 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 50)
        self.fc3 = nn.Linear(50, 10)


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 32 -> 32 -> 16
        x = self.pool(F.relu(self.conv2(x)))  # 16 -> 16 -> 8
        x = self.pool(F.relu(self.conv3(x)))  # 8 -> 8 -> 4
        x = self.pool(F.relu(self.conv4(x)))  # 4 -> 2 -> 1
        #x = self.pool3(F.relu(self.conv3(x)))  # 14 -> 10 -> 5
        x = x.view(-1, 1024 * 1 * 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
        x = self.fc3(x)
        return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# 학습하기
num_epochs = 20
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / i}")

print("학습 완료")

# 테스트하기
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")


Files already downloaded and verified
Files already downloaded and verified
Epoch 1, Loss: 2.1334878715102326
Epoch 2, Loss: 1.6586205039569037
Epoch 3, Loss: 1.3584936102310976
Epoch 4, Loss: 1.1243528506321037
Epoch 5, Loss: 0.9595005229145348
Epoch 6, Loss: 0.8159458035218692
Epoch 7, Loss: 0.731660436353607
Epoch 8, Loss: 0.6548332949439605
Epoch 9, Loss: 0.5979638320052313
Epoch 10, Loss: 0.5458700651515701
Epoch 11, Loss: 0.5047720587683584
Epoch 12, Loss: 0.47616355130094323
Epoch 13, Loss: 0.43380604254339405
Epoch 14, Loss: 0.40342435422904027
Epoch 15, Loss: 0.3745091718459177
Epoch 16, Loss: 0.3486793230554623
Epoch 17, Loss: 0.32464409071601225
Epoch 18, Loss: 0.3077279391114363
Epoch 19, Loss: 0.28510409151623867
Epoch 20, Loss: 0.2649196048896155
학습 완료
Accuracy: 83.62%


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img / 2 + 0.5  # unnormalize, 0~1범위의 이미지를 -1~1의 범위로 사용했기 때문에 -0.5 -> *2
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0))) # (채널, 높이, 너비) -> (높이, 너비, 채널)
    plt.show()

In [ ]:
def visualize_predictions(model, test_loader, num_images=5):
    dataiter = iter(test_loader)
    images, labels = next(dataiter)

    # 이미지 출력
    imshow(torchvision.utils.make_grid(images[:num_images]))

    # 모델 예측
    images = images.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    predicted = predicted.cpu().numpy()

    # 레이블 정의
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    # 예측 결과 출력
    print("Predicted: ", " ".join(classes[predicted[j]] for j in range(num_images)))


In [ ]:
visualize_predictions(model, test_loader, num_images=5)